In [15]:
import tfim
import tfim_perturbation
import tfim_matrices
import numpy as np
from scipy import sparse
from scipy.sparse import linalg as spla
from scipy import linalg
import matplotlib.pyplot as plt
from scipy import optimize
import progressbar
import itertools
import networkx as nx
import matplotlib.pyplot as plt


In [16]:
# Initial system specification
L = [3]
Jij_seed = 55
h_x_range = np.arange(0, 0.005, 0.0001)
h_x_range = np.logspace(-4, -2, 40)

In [17]:
PBC = True
J = 1

In [18]:
# Build lattice and basis
###################################
lattice = tfim.Lattice(L, PBC)
N = lattice.N
basis = tfim.IsingBasis(lattice)
###################################

In [19]:
#construct random J matrix
Jij = tfim.Jij_instance(N,J,"bimodal",Jij_seed) 
Jij = np.array([[-1, 1, 1]])

In [20]:
# List out all the spin_states, corresponding indices and energies
Energies = -tfim.JZZ_SK_ME(basis,Jij)

100% (8 of 8) |##########################| Elapsed Time: 0:00:00 Time:  0:00:00


In [28]:
# Build a list of excited state indices that are one Hamming distances away from ground states
GS_energy, GS_indices = tfim_perturbation.GS(Energies)
for index in range(2**N):
    print(index, basis.state(index), Energies[index])

0 [0 0 0] -1.0
1 [0 0 1] -1.0
2 [0 1 0] 3.0
3 [0 1 1] -1.0
4 [1 0 0] -1.0
5 [1 0 1] 3.0
6 [1 1 0] -1.0
7 [1 1 1] -1.0


In [22]:
def Hamming_array(GS_indices, basis):
    # Calculate Hamming distance array
    Hamming = np.zeros((len(GS_indices), len(GS_indices)))
    for i, n in enumerate(GS_indices):
        basis_n = basis.state(n)
        for j, m in enumerate(GS_indices):
            basis_m = basis.state(m)
            Hamming[i, j] = tfim_perturbation.Hamming_distance(basis_n, basis_m)
    return Hamming

In [23]:
Hamming_matrix = Hamming_array(GS_indices, basis)
print(Hamming_matrix)

[[0. 1. 2. 1. 2. 3.]
 [1. 0. 1. 2. 3. 2.]
 [2. 1. 0. 3. 2. 1.]
 [1. 2. 3. 0. 1. 2.]
 [2. 3. 2. 1. 0. 1.]
 [3. 2. 1. 2. 1. 0.]]


In [24]:
def lst2tuples(coupling_coord):
    #the coupling coord must be len(GS_indices)by 2
    output = []
    for i, coord in enumerate(coupling_coord):
        coord = tuple(coord)
        output.append(coord)
    return output

In [25]:
# Separate out transition matrix

# Construct graph
G = nx.Graph()
G.add_nodes_from(GS_indices)
for order in range(2**N):
    coupling_coord = np.argwhere(Hamming_matrix == order)
    # add edges indicating coupling between ground states
    G.add_edges_from(lst2tuples(GS_indices[coupling_coord]))
    number_connected_components = nx.number_connected_components(G)

In [26]:
# Separate out transition matrix

# Construct graph
G = nx.Graph()
G.add_nodes_from(GS_indices)

# initiate counter
order = 0
number_connected_components = 3

while number_connected_components > 2:
    coupling_coord = np.argwhere(Hamming_matrix == order)
    # add edges indicating coupling between ground states
    G.add_edges_from(lst2tuples(GS_indices[coupling_coord]))
    number_connected_components = nx.number_connected_components(G)
    print(number_connected_components)
    print(order)
    order += 1
# print(order)

6
0
1
1
